# 1. Imports & Setup

In [23]:
import re
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

# 2. Load Excel Data


In [24]:
pathToData = "Freely_quote_data.xlsx"
dataRaw = pd.read_excel(pathToData, sheet_name="Quotes")
dataDefinitions = pd.read_excel(pathToData, sheet_name="Data Dictionary")

# 3. Quick Investigation

In [25]:
dataDefinitions

,Column,Notes
0,destinations,"destinations of travel, can be city, country, ..."
1,trip_start_date,trip departure date
2,trip_end_date,trip return date
3,traveller_ages,number of travellers and individual traveller(...
4,quote_create_time,date and time this quote is generated (between...
5,quote_price,total price of the quote inclusive of boosts c...
6,platform,web: quote from web get a quote path; qw: quot...
7,discount,"discount % applied, noting for a quote with 2 ..."
8,boost_x_name,"extra coverage selected, 9 different extra cov..."
9,boost_x_start_date,extra coverage start date


In [26]:
dataRaw.head(100)

,destinations,trip_start_date,trip_end_date,traveller_ages,quote_create_time,quote_price,platform,discount,boost_1_name,boost_1_start_date,...,boost_6_start_date,boost_6_end_date,boost_7_name,boost_7_start_date,boost_7_end_date,boost_8_name,boost_8_start_date,boost_8_end_date,extra_cancellation,convert
0,Vietnam; Sri Lanka; Portugal; Netherlands; Swi...,30/1/2025,16/10/2025,41;40;11;8;5,2024-24-12 13:20:09,1417,web,0.15,Specified Items,30/1/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NO
1,New Zealand,2024-07-10 00:00:00,14/10/2024,27;25,2024-10-06 11:47:00,79,app,0.15,Adventure Activities,2024-11-10 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,YES
2,All of Europe; Turkey,14/5/2025,2025-08-06 00:00:00,73;73,2024-11-11 12:13:00,516,web,0.15,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO
3,USA,2025-02-01 00:00:00,21/1/2025,45;45;14;13;8,2024-12-12 14:32:00,391,web,0.15,Snow Sports,2025-05-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,NO
4,United Kingdom,30/11/2024,2024-06-12 00:00:00,60,2024-30-11 11:57:29,60,web,0.10,Extra Cancellation,30/11/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,United Kingdom; Spain; Italy,27/3/2025,2025-01-06 00:00:00,65;75,2024-29-12 11:36:01,1261,web,0.15,Extra Cancellation,27/3/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NO
96,Peru; United States of America,24/4/2025,23/5/2025,52,2024-12-07 11:31:00,518,web,0.10,Extra Cancellation,24/4/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NO
97,Japan,2024-09-12 00:00:00,17/12/2024,25,2024-12-09 11:34:00,46,app,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES
98,Domestic Cruise,30/11/2024,2024-07-12 00:00:00,67,2024-27-11 12:42:17,51,web,0.00,Cruise Cover,30/11/2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NO


# 4. Helper: Parse Traveller Ages

In [27]:
def parseAges(agesStr):
    """
    Convert a semicolon-delimited ages string (e.g., "34; 28; 2") into a list of ints.
    Empty/NaN returns an empty list. Non-numeric bits are ignored.
    """
    if pd.isna(agesStr):
        return []
    txt = str(agesStr).strip()
    if not txt or txt.lower() == "nan":
        return []
    out = []
    for part in txt.split(";"):
        part = part.strip()
        if part:
            try:
                out.append(int(part))
            except Exception:
                # ignore badly formed parts quietly
                pass
    return out

dataRaw["travellerAges"] = dataRaw["traveller_ages"].apply(parseAges)
dataRaw["travellerAges"].head()

0     [41, 40, 11, 8, 5]
1               [27, 25]
2               [73, 73]
3    [45, 45, 14, 13, 8]
4                   [60]
Name: travellerAges, dtype: object

# 5. (Not Used) Excess Discount Idea — Left For Reference
Noted negative discounts; keeping note only

This calculates the discount over the pre-applied discount that depends on the number of adult travellers.
- 15% for 2 adults
- 20% for 3+ adults

NOTE: Turns out this is not required. We get negative discounts.

In [28]:
# dataRaw['excessDiscount'] = dataRaw['discount'] - dataRaw['travellerAges'].apply(
#     lambda ages: 0.20 if sum(a >= 18 for a in ages) >= 3
#     else (0.15 if sum(a >= 18 for a in ages) == 2 else 0)
# )

# 6. Date Parsing & Chronology QC (quote / trip start / trip end)
- Robust parsing across multiple formats
- Enforce day-level chronology:
  quoteCreate ≤ tripStart ≤ tripEnd (same-day allowed)
- Keep normalised strings. Flag and preview violations.

In [29]:
# ---------- Formats & Regex ----------
quoteFmt = "%Y-%d-%m %H:%M:%S"  # as specified in your data notes

tripExactFmts = [
    "%Y-%d-%m %H:%M:%S",
    "%Y-%d-%m",
    "%Y-%m-%d %H:%M:%S",
    "%Y-%m-%d",
]

slashDateRe = re.compile(r"^\s*(\d{1,2})/(\d{1,2})/(\d{4})\s*$")


def tryParseExact(text, fmts):
    """
    Try parsing a string against a list of exact datetime formats.
    Returns pandas.Timestamp or None if none match.
    """
    if pd.isna(text):
        return None
    txt = str(text).strip()
    if not txt:
        return None
    for fmt in fmts:
        ts = pd.to_datetime(txt, format=fmt, errors="coerce")
        if pd.notna(ts):
            return ts
    return None


def tripCandidates(text):
    """
    Build candidate parses for a trip date string:
    - Try exact known formats first
    - If it looks like D/M/Y or M/D/Y with slashes, try explicitly both ways
    - Otherwise, try pandas with dayfirst=True/False
    Returns dict with keys 'Y','D','M' mapping to candidate timestamps or None.
    """
    if pd.isna(text):
        return {"Y": None, "D": None, "M": None}
    txt = str(text).strip()
    exact = tryParseExact(txt, tripExactFmts)
    m = slashDateRe.match(txt)
    if m:
        d = pd.to_datetime(txt, format="%d/%m/%Y", errors="coerce")
        m_ = pd.to_datetime(txt, format="%m/%d/%Y", errors="coerce")
        return {
            "Y": exact,
            "D": (d if pd.notna(d) else None),
            "M": (m_ if pd.notna(m_) else None),
        }
    dfirst = pd.to_datetime(txt, errors="coerce", dayfirst=True)
    mfirst = pd.to_datetime(txt, errors="coerce", dayfirst=False)
    return {
        "Y": exact,
        "D": (dfirst if pd.notna(dfirst) else None),
        "M": (mfirst if pd.notna(mfirst) else None),
    }


def dateOnly(ts):
    """
    Floor a timestamp to the date (midnight). None/NaT returns None.
    """
    if ts is None or pd.isna(ts):
        return None
    return ts.floor("D")


def leDay(a, b):
    """
    Day-level comparison: return True if dateOnly(a) ≤ dateOnly(b).
    Treats None/NaT as passing (returns True) to avoid over-flagging in candidate checks.
    """
    if a is None or pd.isna(a) or b is None or pd.isna(b):
        return True
    return dateOnly(a) <= dateOnly(b)


def pickBestTripPair(startCand, endCand, quoteCreate):
    """
    Choose the best (start, end) pair from candidate dicts with day-level constraints:
    - quoteCreate ≤ start ≤ end (same-day allowed)
    Preference order:
      1) Year-first exact for both (Y/Y) if valid
      2) Any valid combo with minimal score (favoring Y over D over M and matching keys)
      3) Fallbacks with violation flagged if no valid combo exists
    Returns (startTs, endTs, note, ambiguousResolved, violatesConstraints).
    """
    # 1) Prefer Y/Y if valid
    if startCand["Y"] is not None and endCand["Y"] is not None:
        sY, eY = startCand["Y"], endCand["Y"]
        violates = not (leDay(quoteCreate, sY) and leDay(sY, eY))
        return (sY, eY, "year-first", False, violates)

    # 2) Score all valid combos
    combos = []
    prefOrder = {"Y": 0, "D": 1, "M": 2}
    for sk, sv in startCand.items():
        if sv is None:
            continue
        for ek, ev in endCand.items():
            if ev is None:
                continue
            if leDay(quoteCreate, sv) and leDay(sv, ev):
                score = prefOrder[sk] + prefOrder[ek] + (0 if sk == ek else 1)
                combos.append((score, sk, ek, sv, ev))
    if combos:
        combos.sort(key=lambda t: (t[0], t[1] != "D", t[2] != "D"))
        best = combos[0]
        bestS, bestE, bestSk, bestEk = best[3], best[4], best[1], best[2]
        ambiguous = any((c[3] != bestS or c[4] != bestE) and c[0] == best[0] for c in combos[1:])
        return (bestS, bestE, f"matched-constraints ({bestSk}/{bestEk})", ambiguous, False)

    # 3) Fallbacks -> mark violation
    if startCand["D"] is not None and endCand["D"] is not None:
        return (startCand["D"], endCand["D"], "fallback-dayfirst", False, True)
    if startCand["M"] is not None and endCand["M"] is not None:
        return (startCand["M"], endCand["M"], "fallback-monthfirst", False, True)

    sVal = next((startCand[k] for k in ("Y", "D", "M") if startCand[k] is not None), None)
    eVal = next((endCand[k] for k in ("Y", "D", "M") if endCand[k] is not None), None)
    return (sVal, eVal, "fallback-mixed", False, True)

# Build working copy
dfDates = dataRaw[["quote_create_time", "trip_start_date", "trip_end_date"]].copy()

# Parse quote_create_time
qcDt = pd.to_datetime(dfDates["quote_create_time"], format=quoteFmt, errors="coerce")
if qcDt.isna().mean() > 0.05:
    qcDt = pd.to_datetime(dfDates["quote_create_time"], errors="coerce", dayfirst=True)
dfDates["quoteCreateDt"] = qcDt

def resolveRowDates(row):
    """
    Resolve per-row trip start/end with constraints, returning:
      tripStartDt, tripEndDt, dateStrategy, dateAmbiguous, dateViolatesConstraints
    """
    sCand = tripCandidates(row.get("trip_start_date"))
    eCand = tripCandidates(row.get("trip_end_date"))
    qc = row.get("quoteCreateDt")
    sDt, eDt, note, ambiguous, violates = pickBestTripPair(sCand, eCand, qc)
    return pd.Series({
        "tripStartDt": sDt,
        "tripEndDt": eDt,
        "dateStrategy": note,
        "dateAmbiguous": bool(ambiguous),
        "dateViolatesConstraints": bool(violates),
    })

resolved = dfDates.apply(resolveRowDates, axis=1)
dfDates = pd.concat([dfDates, resolved], axis=1)

# Day-level checks
qcDay = dfDates["quoteCreateDt"].dt.floor("D")
sDay = dfDates["tripStartDt"].dt.floor("D")
eDay = dfDates["tripEndDt"].dt.floor("D")

anyNat = qcDay.isna() | sDay.isna() | eDay.isna()
flagQcAfterStart = qcDay > sDay
allSameDay = (qcDay == sDay) & (sDay == eDay)
flagQcAfterEnd = (qcDay > eDay) | ((qcDay == eDay) & (~allSameDay))
flagStartAfterEnd = sDay > eDay

dfDates["dateAnyNat"] = anyNat.fillna(True)
dfDates["dateFailQcAfterStart"] = flagQcAfterStart.fillna(False)
dfDates["dateFailQcAfterEnd"] = flagQcAfterEnd.fillna(False)
dfDates["dateFailStartAfterEnd"] = flagStartAfterEnd.fillna(False)

dfDates["dateFailedChecks"] = (
    dfDates["dateAnyNat"]
    | dfDates["dateFailQcAfterStart"]
    | dfDates["dateFailQcAfterEnd"]
    | dfDates["dateFailStartAfterEnd"]
    | dfDates["dateViolatesConstraints"]
)

# Normalised strings
dfDates["tripStartDateNorm"] = dfDates["tripStartDt"].dt.strftime("%Y-%m-%d")
dfDates["tripEndDateNorm"] = dfDates["tripEndDt"].dt.strftime("%Y-%m-%d")
dfDates["quoteCreateTimeNorm"] = dfDates["quoteCreateDt"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Summary
totalRows = len(dfDates)
failedRows = int(dfDates["dateFailedChecks"].sum())
ambigRows = int(dfDates["dateAmbiguous"].sum())
print(f"[Date QC] rows={totalRows} | failures={failedRows} | ambiguousResolved={ambigRows}")

colsShow = [
    "quote_create_time", "trip_start_date", "trip_end_date",
    "quoteCreateDt", "tripStartDt", "tripEndDt",
    "quoteCreateTimeNorm", "tripStartDateNorm", "tripEndDateNorm",
    "dateAnyNat", "dateFailQcAfterStart", "dateFailQcAfterEnd",
    "dateFailStartAfterEnd", "dateViolatesConstraints", "dateAmbiguous",
    "dateStrategy",
]
dateViolations = dfDates.loc[dfDates["dateFailedChecks"], colsShow].copy()
if dateViolations.empty:
    print("No violations found")
else:
    print(f"\nFirst 25 violations (of {failedRows}):")
    display(dateViolations.head(25))

# Write back the three normalised columns only
dataRaw["quote_create_time"] = dfDates["quoteCreateTimeNorm"]
dataRaw["trip_start_date"] = dfDates["tripStartDateNorm"]
dataRaw["trip_end_date"] = dfDates["tripEndDateNorm"]

[Date QC] rows=70000 | failures=0 | ambiguousResolved=0
No violations found


# 7. Booster Date Parsing/Normalisation + QA
- Parse booster start_date / end_date robustly
- Normalise to YYYY-MM-DD and flag issues against trip window

In [30]:
def boosterCandidates(text):
    """
    Build candidates for a booster date:
    1) Try exact formats (including ISO) and return early if success.
    2) If slash-form, try D/M/Y and M/D/Y explicitly.
    3) Else, try pandas with dayfirst True/False.
    Returns dict with keys 'Y','D','M'.
    """
    if pd.isna(text):
        return {"Y": None, "D": None, "M": None}
    txt = str(text).strip()
    if not txt:
        return {"Y": None, "D": None, "M": None}

    exact = tryParseExact(txt, tripExactFmts)
    if exact is not None:
        return {"Y": exact, "D": None, "M": None}

    m = slashDateRe.match(txt)
    if m:
        d = pd.to_datetime(txt, format="%d/%m/%Y", errors="coerce")
        m_ = pd.to_datetime(txt, format="%m/%d/%Y", errors="coerce")
        return {"Y": None, "D": (d if pd.notna(d) else None), "M": (m_ if pd.notna(m_) else None)}

    dfirst = pd.to_datetime(txt, errors="coerce", dayfirst=True)
    mfirst = pd.to_datetime(txt, errors="coerce", dayfirst=False)
    return {"Y": None, "D": (dfirst if pd.notna(dfirst) else None), "M": (mfirst if pd.notna(mfirst) else None)}


def resolveBoostRow(row):
    """
    Resolve one row’s booster start/end:
    - Prefer Y/Y exact
    - Else any valid combo (start ≤ end), prefer within-trip window
    - Else fallback mixed
    Returns s_dt, e_dt, parse_note.
    """
    sCand = boosterCandidates(row["s_raw"])
    eCand = boosterCandidates(row["e_raw"])
    # Prefer exact Y/Y if available
    if sCand["Y"] is not None and eCand["Y"] is not None:
        return pd.Series({"s_dt": sCand["Y"], "e_dt": eCand["Y"], "parse_note": "year-first"})

    combos = []
    prefOrder = {"Y": 0, "D": 1, "M": 2}
    for sk, sv in sCand.items():
        if sv is None:
            continue
        for ek, ev in eCand.items():
            if ev is None:
                continue
            if leDay(sv, ev):
                within = True
                if pd.notna(row["trip_s"]):
                    within &= leDay(row["trip_s"], sv)
                if pd.notna(row["trip_e"]):
                    within &= leDay(ev, row["trip_e"])
                score = prefOrder[sk] + prefOrder[ek] + (0 if sk == ek else 1) + (0 if within else 5)
                combos.append((score, sk, ek, sv, ev))
    if combos:
        combos.sort(key=lambda t: (t[0], t[1] != "D", t[2] != "D"))
        _, sk, ek, sDt, eDt = combos[0]
        return pd.Series({"s_dt": sDt, "e_dt": eDt, "parse_note": f"matched-constraints ({sk}/{ek})"})

    # Fallback mixed
    sVal = next((sCand[k] for k in ("Y", "D", "M") if sCand[k] is not None), None)
    eVal = next((eCand[k] for k in ("Y", "D", "M") if eCand[k] is not None), None)
    return pd.Series({"s_dt": sVal, "e_dt": eVal, "parse_note": "fallback-mixed"})

# Detect booster slots
boostIdx = sorted({
    int(m.group(1))
    for c in dataRaw.columns
    for m in [re.match(r"boost_(\d+)_(start_date|end_date)$", c)]
    if m
})

if not boostIdx:
    print("No booster date columns found.")
else:
    tripS = pd.to_datetime(dataRaw["trip_start_date"], format="%Y-%m-%d", errors="coerce")
    tripE = pd.to_datetime(dataRaw["trip_end_date"], format="%Y-%m-%d", errors="coerce")

    normAssignments = {}
    allViolations = []

    for i in boostIdx:
        sCol = f"boost_{i}_start_date"
        eCol = f"boost_{i}_end_date"
        nCol = f"boost_{i}_name"

        sRaw = dataRaw[sCol] if sCol in dataRaw.columns else pd.Series([pd.NA] * len(dataRaw))
        eRaw = dataRaw[eCol] if eCol in dataRaw.columns else pd.Series([pd.NA] * len(dataRaw))

        tmp = pd.DataFrame({"s_raw": sRaw, "e_raw": eRaw, "trip_s": tripS, "trip_e": tripE})
        resolvedBoost = tmp.apply(resolveBoostRow, axis=1)

        sDay = resolvedBoost["s_dt"].dt.floor("D")
        eDay = resolvedBoost["e_dt"].dt.floor("D")
        tsDay = tripS.dt.floor("D")
        teDay = tripE.dt.floor("D")

        sNorm = sDay.dt.strftime("%Y-%m-%d")
        eNorm = eDay.dt.strftime("%Y-%m-%d")
        normAssignments[sCol] = sNorm
        normAssignments[eCol] = eNorm

        pairPresent = sDay.notna() & eDay.notna()
        startAfterEnd = pairPresent & (sDay > eDay)
        withinTripKnown = pairPresent & tsDay.notna() & teDay.notna()
        startBeforeTrip = withinTripKnown & (sDay < tsDay)
        endAfterTrip = withinTripKnown & (eDay > teDay)

        boostLen = (eDay - sDay).dt.days + 1
        tripLen = (teDay - tsDay).dt.days + 1
        longerThanTrip = withinTripKnown & (boostLen > tripLen)

        failed = startAfterEnd | startBeforeTrip | endAfterTrip | longerThanTrip
        if failed.any():
            viol = pd.DataFrame({
                "rowId": tmp.index,
                "boostIndex": i,
                "name": dataRaw.get(nCol, pd.Series([pd.NA] * len(dataRaw))),
                "startRaw": sRaw, "endRaw": eRaw,
                "startNorm": sNorm, "endNorm": eNorm,
                "tripStart": tsDay.dt.strftime("%Y-%m-%d"),
                "tripEnd": teDay.dt.strftime("%Y-%m-%d"),
                "flagStartAfterEnd": startAfterEnd,
                "flagStartBeforeTrip": startBeforeTrip,
                "flagEndAfterTrip": endAfterTrip,
                "flagLongerThanTrip": longerThanTrip,
                "parseNote": resolvedBoost["parse_note"],
            })
            allViolations.append(viol.loc[failed])

    # Write normalised strings back
    for col, ser in normAssignments.items():
        dataRaw[col] = ser

    violationsAll = pd.concat(allViolations, axis=0, ignore_index=True) if allViolations else pd.DataFrame()
    print(f"[Booster Date QC] boosters={len(boostIdx)} | rows={len(dataRaw)}")
    if violationsAll.empty:
        print("Violations flagged: 0\nNo violations found.")
    else:
        print(f"Violations flagged: {int(violationsAll.shape[0])}")

        flagCols = ["flagStartAfterEnd", "flagStartBeforeTrip", "flagEndAfterTrip", "flagLongerThanTrip"]
        print("\nViolations by type:")
        print(violationsAll[flagCols].sum().sort_values(ascending=False))

        print("\nViolations by booster slot:")
        display(
            violationsAll.groupby("boostIndex")
            .size().rename("violations")
            .reset_index()
            .sort_values("violations", ascending=False)
        )

        desiredCtx = [
            "quote_create_time", "trip_start_date", "trip_end_date",
            "destinations", "platform", "quote_price", "discount", "convert"
        ]
        ctxCols = [c for c in desiredCtx if c in dataRaw.columns]

        def lookupColForRow(row, suffix):
            """
            Helper to fetch slot-specific column from dataRaw for a given violation row and suffix.
            Example suffix: 'name' | 'start_date' | 'end_date'
            """
            col = f"boost_{int(row['boostIndex'])}_{suffix}"
            if col in dataRaw.columns:
                return dataRaw.at[int(row["rowId"]), col]
            return pd.NA

        ctxDf = violationsAll.copy().reset_index(drop=True)
        baseCtx = dataRaw.loc[ctxDf["rowId"], ctxCols].copy().reset_index(drop=True)
        baseCtx.columns = [f"ctx_{c}" for c in baseCtx.columns]
        ctxDf = ctxDf.join(baseCtx)

        ctxDf["ctx_boost_name_in_dataRaw"] = ctxDf.apply(lambda r: lookupColForRow(r, "name"), axis=1)
        ctxDf["ctx_boost_start_in_dataRaw"] = ctxDf.apply(lambda r: lookupColForRow(r, "start_date"), axis=1)
        ctxDf["ctx_boost_end_in_dataRaw"] = ctxDf.apply(lambda r: lookupColForRow(r, "end_date"), axis=1)

        previewCols = [
            "rowId", "boostIndex", "name", "startRaw", "endRaw", "startNorm", "endNorm",
            "tripStart", "tripEnd",
            "flagStartAfterEnd", "flagStartBeforeTrip", "flagEndAfterTrip", "flagLongerThanTrip",
            "parseNote",
            "ctx_boost_name_in_dataRaw", "ctx_boost_start_in_dataRaw", "ctx_boost_end_in_dataRaw",
        ] + [c for c in ctxDf.columns if c.startswith("ctx_")]

        print("\nFirst 25 violations with dataRaw context:")
        display(ctxDf[previewCols].head(25))

[Booster Date QC] boosters=8 | rows=70000
Violations flagged: 228

Violations by type:
flagLongerThanTrip     153
flagEndAfterTrip       138
flagStartBeforeTrip    110
flagStartAfterEnd        0
dtype: int64

Violations by booster slot:


,boostIndex,violations
0,1,143
1,2,58
2,3,19
3,4,8



First 25 violations with dataRaw context:


,rowId,boostIndex,name,startRaw,endRaw,startNorm,endNorm,tripStart,tripEnd,flagStartAfterEnd,...,ctx_trip_start_date,ctx_trip_end_date,ctx_destinations,ctx_platform,ctx_quote_price,ctx_discount,ctx_convert,ctx_boost_name_in_dataRaw,ctx_boost_start_in_dataRaw,ctx_boost_end_in_dataRaw
0,483,1,Gadget Cover,2024-02-12 00:00:00,2024-06-12 00:00:00,2024-12-02,2024-12-06,2024-12-03,2024-12-06,False,...,2024-12-03,2024-12-06,Indonesia,web,93,0.15,NO,Gadget Cover,2024-12-02,2024-12-06
1,2766,1,Existing Medical Condition(s),2024-08-12 00:00:00,2025-02-03 00:00:00,2024-12-08,2025-03-02,2024-12-08,2025-02-28,False,...,2024-12-08,2025-02-28,India,web,370,0.10,YES,Existing Medical Condition(s),2024-12-08,2025-03-02
2,3028,1,Snow Sports,25/12/2024,25/12/2024,2024-12-25,2024-12-25,2024-11-30,2024-12-13,False,...,2024-11-30,2024-12-13,USA,web,161,0.00,NO,Snow Sports,2024-12-25,2024-12-25
3,3469,1,Extra Cancellation,2025-02-05 00:00:00,19/5/2025,2025-05-02,2025-05-19,2025-11-28,2025-12-20,False,...,2025-11-28,2025-12-20,All of Europe,web,221,0.15,NO,Extra Cancellation,2025-05-02,2025-05-19
4,3683,1,Gadget Cover,23/10/2024,15/11/2024,2024-10-23,2024-11-15,2024-10-24,2024-11-15,False,...,2024-10-24,2024-11-15,USA,web,271,0.00,YES,Gadget Cover,2024-10-23,2024-11-15
5,4196,1,Extra Cancellation,22/1/2025,2025-04-07 00:00:00,2025-01-22,2025-07-04,2025-01-22,2025-06-12,False,...,2025-01-22,2025-06-12,All of Europe; All of UK (United Kingdom),web,414,0.00,YES,Extra Cancellation,2025-01-22,2025-07-04
6,4742,1,Extra Cancellation,28/2/2025,2025-02-05 00:00:00,2025-02-28,2025-05-02,2026-01-10,2026-01-10,False,...,2026-01-10,2026-01-10,South West Pacific Cruise,qw,232,0.15,NO,Extra Cancellation,2025-02-28,2025-05-02
7,6564,1,Gadget Cover,2024-12-12 00:00:00,22/2/2025,2024-12-12,2025-02-22,2024-12-12,2024-12-15,False,...,2024-12-12,2024-12-15,Thailand,web,52,0.00,NO,Gadget Cover,2024-12-12,2025-02-22
8,6723,1,Existing Medical Condition(s),19/2/2025,2025-12-03 00:00:00,2025-02-19,2025-03-12,2025-02-19,2025-03-10,False,...,2025-02-19,2025-03-10,Philippines,qw,216,0.15,YES,Existing Medical Condition(s),2025-02-19,2025-03-12
9,7116,1,Existing Medical Condition(s),18/12/2024,22/1/2025,2024-12-18,2025-01-22,2025-12-18,2026-01-24,False,...,2025-12-18,2026-01-24,USA,web,328,0.10,NO,Existing Medical Condition(s),2024-12-18,2025-01-22


# 8. Quote Creation Hour & Day

In [31]:
dataRaw["quoteCreateHour"] = dfDates["quoteCreateDt"].dt.hour
dataRaw["quoteCreateDay"] = dfDates["quoteCreateDt"].dt.day_name()
dataRaw[["quoteCreateHour", "quoteCreateDay"]]


,quoteCreateHour,quoteCreateDay
0,13,Tuesday
1,11,Sunday
2,12,Monday
3,14,Thursday
4,11,Saturday
...,...,...
69995,22,Thursday
69996,13,Wednesday
69997,9,Monday
69998,13,Friday


# 9. Destination → Continent Mapping
- Normalise tokens (aliases, punctuation)
- Map to continents
- Report unmapped tokens

In [32]:
def normToken(s):
    """
    Lowercase, collapse spaces, remove parenthetical parts, normalise hyphens/apostrophes.
    """
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("’", "'").replace("–", "-").replace("—", "-")
    s = re.sub(r"\s*\([^)]*\)", "", s).strip()
    return s

def aliasOrSame(s):
    """
    Return a normalised alias if present; otherwise return the normalised token itself.
    """
    key = normToken(s)
    return ALIASES.get(key, key)

# Aliases (same content as your original; kept intact, only style changed)
ALIASES = {
    "espana": "spain",
    "estados unidos": "united states of america",
    "holland": "netherlands",
    "u.s.a.": "united states of america",
    "usa": "united states of america",
    "america": "united states of america",
    "great britain": "united kingdom",
    "gbr": "united kingdom",
    "all of uk": "united kingdom",
    "all of uk isle of man": "united kingdom",
    "united arab emirates uae": "united arab emirates",
    "united arab emirates u.a.e.": "united arab emirates",
    "hong kong hk": "hong kong",
    "hong kong hongkong": "hong kong",
    "netherlands the netherlands": "netherlands",
    "italy italia": "italy",
    "sri lanka srilanka": "sri lanka",
    "papua new guinea png": "papua new guinea",
    "new zealand nz": "new zealand",
    "korea south": "south korea",
    "korea south republic of korea": "south korea",
    "korea south south korea": "south korea",
    "macau": "macao",
    "malvinas": "falkland islands",
    "herzegovina": "bosnia",
    "kl": "kuala lumpur",
    "port villa": "port vila",
}

AFRICA = {
    "algeria","angola","benin","botswana","burkina faso","cameroon","cape verde",
    "central african republic","chad","egypt","eritrea","ethiopia","gambia","ghana",
    "guinea","ivory coast","cote d'ivoire","kenya","lesotho","liberia","madagascar",
    "malawi","mali","mauritius","morocco","mozambique","namibia","nigeria","rwanda",
    "senegal","seychelles","sierra leone","south africa","south sudan","swaziland",
    "eswatini","tanzania","togo","tunisia","uganda","zambia","zimbabwe","reunion",
    "cairo","cape town","johannesburg","marrakech","nairobi",
}

ASIA = {
    "afghanistan","armenia","azerbaijan","bahrain","bangladesh","bhutan","brunei",
    "cambodia","china","cyprus","east timor","timor-leste","georgia","hong kong",
    "india","indonesia","iran","iraq","israel","japan","jordan","kazakhstan","kuwait",
    "kyrgyzstan","laos","lebanon","macao","malaysia","maldives","mongolia","myanmar",
    "burma","nepal","oman","pakistan","philippines","qatar","saudi arabia","singapore",
    "south korea","sri lanka","syria","taiwan","tajikistan","thailand","turkey",
    "turkmenistan","united arab emirates","uzbekistan","vietnam","yemen",
    "abu dhabi","bangkok","beijing","chiang mai","delhi","dhaka","doha","dubai",
    "hanoi","ho chi minh city","istanbul","jakarta","kuala lumpur","kathmandu",
    "osaka","sapporo","seoul","shanghai","shenzhen","siem reap","taipei","tokyo",
    "penang","phuket","koh samui","denpasar","kuta","ubud","manila","cebu","mumbai",
    "bali","jordan",
}

EUROPE = {
    "albania","andorra","austria","belgium","bosnia","bulgaria","croatia",
    "czech republic","denmark","estonia","finland","france","germany","gibraltar",
    "greece","guernsey","hungary","iceland","republic of ireland","ireland","italy",
    "jersey","kosovo","latvia","liechtenstein","lithuania","luxembourg","macedonia",
    "malta","moldova","monaco","montenegro","netherlands","norway","poland",
    "portugal","romania","san marino","serbia","slovakia","slovenia","spain",
    "sweden","switzerland","united kingdom","england","scotland","wales",
    "northern ireland","vatican city","faroe islands","canary islands","madeira",
    "lapland","iceland",
    "amsterdam","athens","barcelona","belgrade","berlin","brussels","budapest",
    "cologne","copenhagen","dublin","dubrovnik","edinburgh","florence","frankfurt",
    "geneva","glasgow","glasglow","helsinki","ibiza","lisbon","liverpool","london",
    "lyon","madrid","milan","munich","nice","paris","prague","rome","salzburg",
    "stockholm","strasbourg","venice","vienna","zurich","nuremburg","nuremberg",
    "brussels","bratislava",
}

NORTH_AMERICA = {
    "canada","united states of america","united states","mexico","greenland",
    "alabama","alaska","arizona","california","colorado","connecticut","florida",
    "indiana","massachusetts","michigan","minnesota","nebraska","new jersey",
    "new mexico","new york","north carolina","oregon","tennessee","texas","utah",
    "virginia","hawaii",
    "alberta","british columbia","nova scotia","quebec",
    "chicago","honolulu","las vegas","los angeles","miami","san francisco",
    "new york","ottawa","toronto","vancouver","calgary","quebec","mexico city",
    "cancun",
    "belize","costa rica","el salvador","guatemala","honduras","nicaragua","panama",
    "panama city","bahamas","barbados","bermuda","cayman islands","cuba","dominica",
    "dominican rep.","guadeloupe","haiti","jamaica","martinique","puerto rico",
    "st. lucia","st. kitts-nevis","trinidad and tobago","antigua and barbuda",
    "aruba","anguilla","bermuda","virgin islands",
}

SOUTH_AMERICA = {
    "argentina","bolivia","brazil","chile","colombia","ecuador","guyana","paraguay",
    "peru","uruguay","venezuela","falkland islands",
    "buenos aires","rio de janeiro","santiago","lima",
}

OCEANIA = {
    "australia","new zealand","fiji","vanuatu","new caledonia","papua new guinea",
    "nauru","palau","samoa","western samoa","solomon islands","tonga",
    "french polynesia","tahiti","cook islands","rarotonga","american samoa",
    "norfolk island","lord howe island",
    "queensland","western australia","south australia","northern territory","tasmania",
    "great barrier reef","whitsundays","hamilton island","fraser island",
    "isle of pines","lifou","loyalty islands","port vila","port vila","suva","nadi",
    "noumea","mystery island",
    "sydney","melbourne","brisbane","perth","adelaide","hobart","canberra",
    "gold coast","sunshine coast","noosa","port douglas","cairns","broome",
    "alice springs","uluru","airlie beach","auckland","christchurch","queenstown",
    "domestic cruise","australia domestic cruise","south west pacific cruise",
    "new zealand",
}

ANTARCTICA = {"antarctica","antarctica sightseeing flight","antarctica cruising"}

AGGREGATES = {
    "all of africa": {"Africa"},
    "all of asia": {"Asia"},
    "all of europe": {"Europe"},
    "all of europe scandinavia": {"Europe"},
    "all of uk": {"Europe"},
    "all of north america": {"North America"},
    "all of south america": {"South America"},
    "all of south america patagonia": {"South America"},
    "all of the americas": {"North America", "South America"},
    "all of the americas central america": {"North America"},
    "all of the pacific": {"Oceania"},
    "all of the pacific pacific islands": {"Oceania"},
    "all of the middle east": {"Asia"},
    "worldwide": {"Worldwide"},
    "south west pacific cruise": {"Oceania"},
    "domestic cruise": {"Oceania"},
    "australia domestic cruise": {"Oceania"},
}

def lookupContinents(token):
    """
    Map a single normalised destination token to a set of continent labels.
    Returns an empty set if unknown.
    """
    t = aliasOrSame(token)

    if t in AGGREGATES:
        return AGGREGATES[t]
    if t in AFRICA: return {"Africa"}
    if t in ASIA: return {"Asia"}
    if t in EUROPE: return {"Europe"}
    if t in NORTH_AMERICA: return {"North America"}
    if t in SOUTH_AMERICA: return {"South America"}
    if t in OCEANIA: return {"Oceania"}
    if t in ANTARCTICA: return {"Antarctica"}

    if " middle east" in t:
        return {"Asia"}
    if t.endswith(" islands"):
        if t in {"canary islands", "madeira", "faroe islands"}:
            return {"Europe"}
    return set()

def mapRowToContinents(destinations):
    """
    For a semicolon-separated 'destinations' string, return a semicolon-joined, sorted set of continents.
    Returns 'Unknown' if no token maps to a continent.
    """
    if pd.isna(destinations) or not str(destinations).strip():
        return ""
    tokens = [d.strip() for d in str(destinations).split(";") if d.strip()]
    out = set()
    for tok in tokens:
        out |= lookupContinents(tok)
    if not out:
        return "Unknown"
    return ";".join(sorted(out))

# Count & list destinations (kept from your original flow)
allDestinations = dataRaw["destinations"].str.split(";").explode().str.strip().value_counts()

# Unique tokens for coverage check
uniqueDestinations = dataRaw["destinations"].str.split(";").explode().str.strip().unique()
sorted(uniqueDestinations)

print(len(uniqueDestinations))

# Create continent column (camelCase)
dataRaw["destinationContinent"] = dataRaw["destinations"].apply(mapRowToContinents)

# Coverage report for your uniqueDestinations
normedUnique = sorted({aliasOrSame(x) for x in uniqueDestinations})
unmapped = [u for u in normedUnique if not lookupContinents(u)]
print(f"Unmapped tokens ({len(unmapped)}). Add them if you see any you care about:\n", unmapped[:50])
dataRaw[["destinations", "destinationContinent"]].head(10)

# Create 1/0 indicator columns for each continent based on `destinationContinent`
continentLabels = ["Africa", "Asia", "Europe", "North America", "South America", "Oceania", "Antarctica"]

for label in continentLabels:
    colName = "is" + label.replace(" ", "")          # e.g., "North America" -> "isNorthAmerica"
    pattern = rf"(?:^|;){re.escape(label)}(?:;|$)"   # match whole label in the semicolon list
    dataRaw[colName] = dataRaw["destinationContinent"].str.contains(pattern, na=False).astype(int)

# Quick peek
dataRaw[["destinations", "destinationContinent"] + ["isAfrica","isAsia","isEurope","isNorthAmerica","isSouthAmerica","isOceania","isAntarctica"]].head(10)


407
Unmapped tokens (14). Add them if you see any you care about:
 ['antarctica-sightseeing flight', 'birmingham', 'brasilia', 'cardiff', 'darwin', 'kangaroo island', 'korea', 'lombok', 'manchester', 'netherlands antilles', 'pattaya', 'salvador', 'tenerife', 'tibet']


,destinations,destinationContinent,isAfrica,isAsia,isEurope,isNorthAmerica,isSouthAmerica,isOceania,isAntarctica
0,Vietnam; Sri Lanka; Portugal; Netherlands; Swi...,Asia;Europe,0,1,1,0,0,0,0
1,New Zealand,Oceania,0,0,0,0,0,1,0
2,All of Europe; Turkey,Asia;Europe,0,1,1,0,0,0,0
3,USA,North America,0,0,0,1,0,0,0
4,United Kingdom,Europe,0,0,1,0,0,0,0
5,Croatia,Europe,0,0,1,0,0,0,0
6,Indonesia,Asia,0,1,0,0,0,0,0
7,New Zealand,Oceania,0,0,0,0,0,1,0
8,Greece; Turkey; Malta,Asia;Europe,0,1,1,0,0,0,0
9,Cook Islands,Oceania,0,0,0,0,0,1,0


# 10. Quote Time-of-Day Buckets (Dayparts)

In [33]:
binsDaypart = [-1, 5, 11, 17, 23]  # 0..23 covered
labelsDaypart = ["Night", "Morning", "Afternoon", "Evening"]

cutDaypart = pd.cut(
    dataRaw["quoteCreateHour"],
    bins=binsDaypart,
    labels=labelsDaypart,
    include_lowest=True
)

catType = CategoricalDtype(categories=labelsDaypart + ["Unknown"], ordered=True)
dataRaw["quoteCreateDaypart"] = cutDaypart.cat.add_categories("Unknown").fillna("Unknown").astype(catType)

print(dataRaw["quoteCreateDaypart"].value_counts(dropna=False))

quoteCreateDaypart
Afternoon    29029
Evening      20079
Morning      17864
Night         3028
Unknown          0
Name: count, dtype: int64


# 11. Lead Time (days from quote to trip start)

In [34]:
qcDayParsed = pd.to_datetime(dataRaw["quote_create_time"], format="%Y-%m-%d %H:%M:%S", errors="coerce").dt.floor("D")
startDayParsed = pd.to_datetime(dataRaw["trip_start_date"], format="%Y-%m-%d", errors="coerce")

dataRaw["leadTimeDays"] = (startDayParsed - qcDayParsed).dt.days.astype("Int64")

print("Lead time (days) summary:")
print(dataRaw["leadTimeDays"].describe())
display(dataRaw[["quote_create_time", "trip_start_date", "leadTimeDays"]].head(10))

Lead time (days) summary:
count      70000.0
mean     60.175971
std      79.942187
min            0.0
25%            7.0
50%           25.0
75%           83.0
max          547.0
Name: leadTimeDays, dtype: Float64


,quote_create_time,trip_start_date,leadTimeDays
0,2024-12-24 13:20:09,2025-01-30,37
1,2024-10-06 11:47:00,2024-10-07,1
2,2024-11-11 12:13:00,2025-05-14,184
3,2024-12-12 14:32:00,2025-01-02,21
4,2024-11-30 11:57:29,2024-11-30,0
5,2024-12-02 15:35:00,2025-09-07,279
6,2024-12-08 09:06:00,2024-12-23,15
7,2024-12-22 15:08:10,2024-12-24,2
8,2024-10-08 17:01:00,2025-03-26,169
9,2024-12-31 11:51:41,2025-01-07,7


# 12. Trip Length (days, inclusive)

In [35]:
startParsed = pd.to_datetime(dataRaw["trip_start_date"], format="%Y-%m-%d", errors="coerce")
endParsed = pd.to_datetime(dataRaw["trip_end_date"], format="%Y-%m-%d", errors="coerce")

tripLen = (endParsed - startParsed).dt.days + 1
tripLen = tripLen.where(startParsed.notna() & endParsed.notna(), other=pd.NA)
tripLen = tripLen.where(tripLen >= 1, other=pd.NA)

dataRaw["tripLengthDays"] = tripLen.astype("Int64")

print("Trip length (days) summary:")
print(dataRaw["tripLengthDays"].describe())
display(dataRaw[["trip_start_date", "trip_end_date", "tripLengthDays"]].head(10))

Trip length (days) summary:
count      70000.0
mean       28.0224
std      48.395715
min            1.0
25%           10.0
50%           16.0
75%           29.0
max          548.0
Name: tripLengthDays, dtype: Float64


,trip_start_date,trip_end_date,tripLengthDays
0,2025-01-30,2025-10-16,260
1,2024-10-07,2024-10-14,8
2,2025-05-14,2025-06-08,26
3,2025-01-02,2025-01-21,20
4,2024-11-30,2024-12-06,7
5,2025-09-07,2025-11-13,68
6,2024-12-23,2025-01-01,10
7,2024-12-24,2025-01-07,15
8,2025-03-26,2025-04-27,33
9,2025-01-07,2025-01-14,8


# 13. Number of Travellers

In [36]:
def countTravellers(x):
    """
    Count number of travellers from parsed list; if list missing, count non-empty ';'-separated parts.
    """
    if isinstance(x, list):
        return len(x)
    if pd.isna(x) or not str(x).strip():
        return 0
    return sum(1 for part in str(x).split(";") if part.strip())

dataRaw["numTravellers"] = dataRaw["travellerAges"].apply(countTravellers).astype("Int64")

print("numTravellers value counts:")
print(dataRaw["numTravellers"].value_counts(dropna=False).sort_index())
display(dataRaw[["traveller_ages", "travellerAges", "numTravellers"]].head(10))

numTravellers value counts:
numTravellers
1    32721
2    25565
3     4613
4     4990
5     1455
6      426
7      144
8       86
Name: count, dtype: Int64


,traveller_ages,travellerAges,numTravellers
0,41;40;11;8;5,"[41, 40, 11, 8, 5]",5
1,27;25,"[27, 25]",2
2,73;73,"[73, 73]",2
3,45;45;14;13;8,"[45, 45, 14, 13, 8]",5
4,60,[60],1
5,52;43,"[52, 43]",2
6,25,[25],1
7,45;46;17;15,"[45, 46, 17, 15]",4
8,81;78,"[81, 78]",2
9,25,[25],1


# 14. Age Cohorts (baby / child / senior)

In [37]:
INFANT_MAX = 1
CHILD_MIN, CHILD_MAX = 2, 17
SENIOR_MIN = 65

def countIf(ages, pred):
    """
    Count ages satisfying predicate `pred(age)`. Non-list input returns 0.
    """
    if not isinstance(ages, list):
        return 0
    c = 0
    for a in ages:
        try:
            ai = int(a)
        except Exception:
            continue
        if pred(ai):
            c += 1
    return c

dataRaw["babyCount"] = dataRaw["travellerAges"].apply(lambda ages: countIf(ages, lambda a: a <= INFANT_MAX)).astype("Int64")
dataRaw["childCount"] = dataRaw["travellerAges"].apply(lambda ages: countIf(ages, lambda a: CHILD_MIN <= a <= CHILD_MAX)).astype("Int64")
dataRaw["seniorCount"] = dataRaw["travellerAges"].apply(lambda ages: countIf(ages, lambda a: a >= SENIOR_MIN)).astype("Int64")

dataRaw["hasBaby"] = (dataRaw["babyCount"] > 0).astype("boolean")
dataRaw["hasChild"] = (dataRaw["childCount"] > 0).astype("boolean")
dataRaw["hasSenior"] = (dataRaw["seniorCount"] > 0).astype("boolean")

print("Baby / Child / Senior counts:")
print(dataRaw[["babyCount", "childCount", "seniorCount"]].describe())
display(
    dataRaw[["traveller_ages", "travellerAges", "babyCount", "childCount", "seniorCount", "hasBaby", "hasChild", "hasSenior"]].head(10)
)


Baby / Child / Senior counts:
       babyCount  childCount  seniorCount
count    70000.0     70000.0      70000.0
mean    0.012114    0.231171     0.430414
std     0.110437     0.63412     0.712456
min          0.0         0.0          0.0
25%          0.0         0.0          0.0
50%          0.0         0.0          0.0
75%          0.0         0.0          1.0
max          2.0         6.0          6.0


,traveller_ages,travellerAges,babyCount,childCount,seniorCount,hasBaby,hasChild,hasSenior
0,41;40;11;8;5,"[41, 40, 11, 8, 5]",0,3,0,False,True,False
1,27;25,"[27, 25]",0,0,0,False,False,False
2,73;73,"[73, 73]",0,0,2,False,False,True
3,45;45;14;13;8,"[45, 45, 14, 13, 8]",0,3,0,False,True,False
4,60,[60],0,0,0,False,False,False
5,52;43,"[52, 43]",0,0,0,False,False,False
6,25,[25],0,0,0,False,False,False
7,45;46;17;15,"[45, 46, 17, 15]",0,2,0,False,True,False
8,81;78,"[81, 78]",0,0,2,False,False,True
9,25,[25],0,0,0,False,False,False


# 15. Weekend-Only Trip Flag
- 1-day starting Sat/Sun, or 2-day Sat/Sun

Dont like this. Come back to this.

In [38]:
startDOW = startParsed.dt.dayofweek  # Mon=0 ... Sun=6
endDOW = endParsed.dt.dayofweek

isWeekendOnly = (
    startParsed.notna() & endParsed.notna() & (
        ((dataRaw["tripLengthDays"] == 1) & startDOW.isin([5, 6])) |
        ((dataRaw["tripLengthDays"] == 2) & (startDOW == 5) & (endDOW == 6))
    )
)

dataRaw["isWeekendTrip"] = isWeekendOnly.astype("boolean")

print(dataRaw["isWeekendTrip"].value_counts(dropna=False))
display(dataRaw[["trip_start_date", "trip_end_date", "tripLengthDays", "isWeekendTrip"]].head(10))

isWeekendTrip
False    69621
True       379
Name: count, dtype: Int64


,trip_start_date,trip_end_date,tripLengthDays,isWeekendTrip
0,2025-01-30,2025-10-16,260,False
1,2024-10-07,2024-10-14,8,False
2,2025-05-14,2025-06-08,26,False
3,2025-01-02,2025-01-21,20,False
4,2024-11-30,2024-12-06,7,False
5,2025-09-07,2025-11-13,68,False
6,2024-12-23,2025-01-01,10,False
7,2024-12-24,2025-01-07,15,False
8,2025-03-26,2025-04-27,33,False
9,2025-01-07,2025-01-14,8,False


# 16. Solo Flag

In [39]:
dataRaw["isSoloTraveller"] = (dataRaw["numTravellers"] == 1).astype("boolean")

display(dataRaw["isSoloTraveller"].head(15))

0     False
1     False
2     False
3     False
4      True
5     False
6      True
7     False
8     False
9      True
10     True
11    False
12    False
13     True
14    False
Name: isSoloTraveller, dtype: boolean

# 17. Pricing Normals

In [40]:
priceNum = pd.to_numeric(dataRaw["quote_price"], errors="coerce")
daysNum = pd.to_numeric(dataRaw["tripLengthDays"], errors="coerce")
paxNum = pd.to_numeric(dataRaw["numTravellers"], errors="coerce")
paxDays = paxNum * daysNum

with np.errstate(divide="ignore", invalid="ignore"):
    dataRaw["pricePerDay"] = (priceNum / daysNum).astype("Float64")
    dataRaw["pricePerTraveller"] = (priceNum / paxNum).astype("Float64")
    dataRaw["pricePerTravellerPerDay"] = (priceNum / paxDays).astype("Float64")

for c in ["pricePerDay", "pricePerTraveller", "pricePerTravellerPerDay"]:
    dataRaw[c] = dataRaw[c].round(2)

print("Pricing normals preview:")
colsShow = [
    "quote_price", "tripLengthDays", "numTravellers",
    "pricePerDay", "pricePerTraveller", "pricePerTravellerPerDay"
]
display(dataRaw[colsShow].head(15))

print("\nBasic stats:")
print(dataRaw[["pricePerDay", "pricePerTraveller", "pricePerTravellerPerDay"]].describe())

Pricing normals preview:


,quote_price,tripLengthDays,numTravellers,pricePerDay,pricePerTraveller,pricePerTravellerPerDay
0,1417,260,5,5.45,283.4,1.09
1,79,8,2,9.88,39.5,4.94
2,516,26,2,19.85,258.0,9.92
3,391,20,5,19.55,78.2,3.91
4,60,7,1,8.57,60.0,8.57
5,711,68,2,10.46,355.5,5.23
6,78,10,1,7.8,78.0,7.8
7,134,15,4,8.93,33.5,2.23
8,1479,33,2,44.82,739.5,22.41
9,43,8,1,5.38,43.0,5.38



Basic stats:
       pricePerDay  pricePerTraveller  pricePerTravellerPerDay
count      70000.0            70000.0                  70000.0
mean     18.797319          236.94473                12.188073
std       32.20779         488.662307                23.080992
min           0.84                6.0                      0.5
25%           7.55               69.0                     4.82
50%           11.4              122.0                     6.94
75%           19.0           243.0825                    11.42
max         1467.0            45011.0                   1467.0


# 18. Booster One-Hot + Count per Quote
- Create canonical booster columns (0/1)
- Sum to numBoostersApplied

In [41]:
CANON_BOOSTERS = [
    "Extra Cancellation",
    "Cruise Cover",
    "Snow Sports",
    "Existing Medical Condition(s)",
    "Gadget Cover",
    "Motorcycle Cover",
    "Rental Vehicle Insurance Excess",
    "Adventure Activities",
    "Specified Items",
]

def cleanBoostName(s):
    """
    Normalise booster label for robust matching:
    - trim
    - collapse whitespace
    - normalise punctuation
    - lowercase
    """
    if pd.isna(s):
        return ""
    s = str(s).strip()
    if not s:
        return ""
    s = re.sub(r"\s+", " ", s)
    s = s.replace("–", "-").replace("—", "-").replace("’", "'")
    return s.lower()

def slugBoost(s):
    """
    Turn a human-readable booster label into a safe column suffix.
    Remove parentheses; keep alphanumerics; convert separators to underscores.
    """
    s = re.sub(r"[()]", "", s)
    s = re.sub(r"[^0-9A-Za-z]+", "_", s).strip("_")
    return s

boostNameCols = sorted(
    [c for c in dataRaw.columns if re.fullmatch(r"boost_\d+_name", c)],
    key=lambda x: int(re.search(r"boost_(\d+)_name", x).group(1))
)

if not boostNameCols:
    print("No booster name columns found (boost_*_name). Skipping one-hot creation.")
else:
    namesCleanDf = pd.DataFrame({c: dataRaw[c].map(cleanBoostName) for c in boostNameCols})

    createdCols = []
    for label in CANON_BOOSTERS:
        target = cleanBoostName(label)  # canonical lowercase
        newCol = f"boost_{slugBoost(label)}"
        dataRaw[newCol] = namesCleanDf.eq(target).any(axis=1).astype("int8")
        createdCols.append(newCol)

    dataRaw["numBoostersApplied"] = dataRaw[createdCols].sum(axis=1).astype("Int64")

    print("Created booster one-hot columns:")
    print(createdCols)
    print("\nOne-hot column sums (number of quotes with that booster at least once):")
    print(dataRaw[createdCols].sum().sort_values(ascending=False))
    display(dataRaw[createdCols]  # compact
            .assign(numBoostersApplied=dataRaw["numBoostersApplied"])
            .head(10))

Created booster one-hot columns:
['boost_Extra_Cancellation', 'boost_Cruise_Cover', 'boost_Snow_Sports', 'boost_Existing_Medical_Conditions', 'boost_Gadget_Cover', 'boost_Motorcycle_Cover', 'boost_Rental_Vehicle_Insurance_Excess', 'boost_Adventure_Activities', 'boost_Specified_Items']

One-hot column sums (number of quotes with that booster at least once):
boost_Extra_Cancellation                 17122
boost_Cruise_Cover                        7400
boost_Snow_Sports                         4477
boost_Existing_Medical_Conditions         2355
boost_Gadget_Cover                        2277
boost_Motorcycle_Cover                    1671
boost_Rental_Vehicle_Insurance_Excess     1069
boost_Adventure_Activities                 566
boost_Specified_Items                      519
dtype: int64


,boost_Extra_Cancellation,boost_Cruise_Cover,boost_Snow_Sports,boost_Existing_Medical_Conditions,boost_Gadget_Cover,boost_Motorcycle_Cover,boost_Rental_Vehicle_Insurance_Excess,boost_Adventure_Activities,boost_Specified_Items,numBoostersApplied
0,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,1,0,1
2,0,0,0,0,0,0,0,0,0,0
3,1,0,1,1,0,0,0,0,0,3
4,1,0,0,0,0,0,0,0,0,1
5,1,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,1,0,0,1,0,0,0,0,0,2
9,0,0,0,0,0,0,0,0,0,0


# 19. Final Columns

In [42]:
dataRaw.columns

Index(['destinations', 'trip_start_date', 'trip_end_date', 'traveller_ages',
       'quote_create_time', 'quote_price', 'platform', 'discount',
       'boost_1_name', 'boost_1_start_date', 'boost_1_end_date',
       'boost_2_name', 'boost_2_start_date', 'boost_2_end_date',
       'boost_3_name', 'boost_3_start_date', 'boost_3_end_date',
       'boost_4_name', 'boost_4_start_date', 'boost_4_end_date',
       'boost_5_name', 'boost_5_start_date', 'boost_5_end_date',
       'boost_6_name', 'boost_6_start_date', 'boost_6_end_date',
       'boost_7_name', 'boost_7_start_date', 'boost_7_end_date',
       'boost_8_name', 'boost_8_start_date', 'boost_8_end_date',
       'extra_cancellation', 'convert', 'travellerAges', 'quoteCreateHour',
       'quoteCreateDay', 'destinationContinent', 'isAfrica', 'isAsia',
       'isEurope', 'isNorthAmerica', 'isSouthAmerica', 'isOceania',
       'isAntarctica', 'quoteCreateDaypart', 'leadTimeDays', 'tripLengthDays',
       'numTravellers', 'babyCount', 'chil

# 20. Save updated Data

In [43]:
dataRaw.to_csv("cleaned_data.csv", index=False)